In [4]:
import requests
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
from time import sleep
from random import randint

# API Camp to Camp - 

In [67]:
#Export data from C2C API

url = 'https://api.camptocamp.org/outings'
france = '&a=14274'
req_activities = {'?act=skitouring':310,'?act=rock_climbing':330,'?act=mountain_climbing':100,'?act=snow_ice_mixed':100,'?act=ice_climbing':100,'?act=hiking':100}

request_num = 0
result=[]

for k,v in req_activities.items():
    for i in range(0,v):
        if i == 0 : 
            api_path = url + k + france
        else:
            api_path = url + k + france + '&offset=' + str(i*30)            
        r = requests.get(api_path)
        result.append(r.json())
        request_num += 1
        sleep(randint(1,3))
        
print("Nombre total de reqûetes :" + str(request_num))

Nombre total de reqûetes :3


# # Build dataframe & export to csv - 

In [71]:
# Flatten json file - 
d = pd.DataFrame()

arr_lat=[]
arr_long=[]

for page in result:    
    
    d=d.append(json_normalize(page['documents'],max_level=1))
    
    for outing in page['documents']:
        arr_lat.append(json.loads(outing['geometry']['geom'])['coordinates'][0])
        arr_long.append(json.loads(outing['geometry']['geom'])['coordinates'][1])
            
d.reset_index(inplace=True, drop=True)

# Select relevant columns -  
cols = ['document_id','activities','date_start','date_end','author.name','author.user_id']
d=d[cols]

#Add geo coordinates
d['latitude']=arr_lat
d['longitude']=arr_long

# Print first rows
d.head()

,document_id,activities,date_start,date_end,author.name,author.user_id,latitude,longitude
0,1196222,[skitouring],2020-03-23,2020-03-23,christian73,1071965,722846.725591,5.729444e+06
1,1195244,"[skitouring, snow_ice_mixed, mountain_climbing...",2020-03-17,2020-03-17,dormeur10,800213,773581.331208,5.770432e+06
2,1195133,[skitouring],2020-03-17,2020-03-17,christian73,1071965,728011.892440,5.728002e+06
3,1195132,[skitouring],2020-03-17,2020-03-17,janic,578085,19054.692104,5.294350e+06
4,1195116,"[skitouring, snow_ice_mixed, paragliding]",2020-03-17,2020-03-17,Paulo,991164,723782.408575,5.766919e+06


In [72]:
# Clean columns -

# Clean activity - 
def dummies_activity(x,a):
    if a in x:
        return True
    else:
        return False

activities=['skitouring', 'rock_climbing','mountain_climbing','ice_climbing','hiking','snow_ice_mixed']

for activity in activities:
    d[activity]=d.activities.apply(lambda x: dummies_activity(x,activity))

d.drop('activities', inplace=True, axis=1)

# Drop_duplicates (caused by multi-activities outings exported several times during requests)
d.drop_duplicates(inplace=True)

# Convert to datetime
col_to_date=['date_end','date_start']
d[col_to_date]=d[col_to_date].apply(lambda x: pd.to_datetime(x))

# Convert to integer
col_to_int=['document_id','author.user_id']
d[col_to_int]=d[col_to_int].astype(int)

# Export to csv -
d.to_csv('outings.csv',index=False)

# CHECK - 
d.head()

,document_id,date_start,date_end,author.name,author.user_id,latitude,longitude,skitouring,rock_climbing,mountain_climbing,ice_climbing,hiking,snow_ice_mixed
0,1196222,2020-03-23,2020-03-23,christian73,1071965,722846.725591,5.729444e+06,True,False,False,False,False,False
1,1195244,2020-03-17,2020-03-17,dormeur10,800213,773581.331208,5.770432e+06,True,False,True,True,False,True
2,1195133,2020-03-17,2020-03-17,christian73,1071965,728011.892440,5.728002e+06,True,False,False,False,False,False
3,1195132,2020-03-17,2020-03-17,janic,578085,19054.692104,5.294350e+06,True,False,False,False,False,False
4,1195116,2020-03-17,2020-03-17,Paulo,991164,723782.408575,5.766919e+06,True,False,False,False,False,True
